# AsTRiQue Showcase

This is a showcase of [AsTRiQue](https://github.com/prokophanzl/astrique), an active machine learning framework for reducing stimulus count in perception experiments. In it, an oracle will be repeatedly queried to classify sounds as either s/ʃ or z/ʒ (/s/ and /ʃ/ are batched under /s/, /z/ and /ʒ/ are batched under /z/). Once the model has good enough predictions for the rest of the stimuli, its performance will be evaluated on the sounds the oracle didn't classify and a chart will be plotted displaying the oracle's answers and the model's predictions.

In virtual agent mode, a lookup table (virtual agent) will be used as the oracle. In live participant mode, you will play the role of the oracle yourself.


In [ ]:
# @title Technical Setup
# @markdown Run this cell to resolve imports and Google Colab-specific features (even if you are not running this notebook in Google Colab).

IN_COLAB = os.getenv("COLAB_RELEASE_TAG") is not None
if IN_COLAB:
	print("Notebook is running in Google Colab. Getting data...")
	
	# check if data has already been downloaded - check for data directory
	if not os.path.exists('data'):
		!git clone https://github.com/prokophanzl/AsTRiQue/
		!cd AsTRiQue
		print("Data downloaded.")
	else:
		print("Data already downloaded.")

	print("Getting dependencies...")
	%pip install jupyter_ui_poll
	from IPython.display import Audio, display
else:
	print("Notebook is not running in Google Colab; skipping data download.")
	from playsound import playsound

import os
import astrique_module
import time
from ipywidgets import Button, HBox, VBox
from jupyter_ui_poll import ui_events

In [ ]:
# @title Config

config = astrique_module.Config()

# resource setup (do not change for the showcase)
config.PREDICTORS = ("voicing", "duration")
config.FILENAME_COL = "filename"
config.LABEL_MAPPING = {"s": 0, "z": 1}
config.TARGET = "answer_batch"
config.DATA_PATH = "data/data.csv"
config.USE_VIRTUAL_AGENT = False
config.PARTICIPANT_CSV_DIR = "data/participants"
config.PROCESSED_PATH = "data_processed.csv"

# @markdown In this cell, you can change the following config values:
# @markdown <hr>

config.STRATIFIED_SAMPLING_RESOLUTION = (
    3  # @param {type:"slider", min: 1, max: 10, step: 1}
)
# @markdown resolution for stratified sampling of random samples; number of initial stratified samples collected is up to STRATIFIED_SAMPLING_RESOLUTION^2
# @markdown <hr>

config.MIN_ITERATIONS = 30  # @param {type:"slider", min: 0, max: 104, step: 1}
# @markdown minimum number of iterations
# @markdown <hr>

config.MAX_ITERATIONS = 0  # @param {type:"slider", min: 0, max: 104, step: 1}
# @markdown maximum number of iterations; 0 to disable
# @markdown <hr>

config.CLEANSER_FREQUENCY = 0  # @param {type:"slider", min: 0, max: 20, step: 1}
# @markdown insert a high-certainty sample every nth iteration to prevent participant fatigue (irrelevant for virtual agents); 0 to disable
# @markdown <hr>

config.MODEL_CERTAINTY_CUTOFF = (
    0.95  # @param {type:"slider", min: 0.5 , max: 1, step: 0.01}
)
# @markdown certainty threshold to end training
# @markdown <hr>

config.DEBUG_MODE = True  # @param {type:"boolean"}
# @markdown print info messages during the program's runtime
# @markdown <hr>

config.RNG_SEED = 42  # @param {type:"slider", min: 0, max: 100, step: 1}
# @markdown seed for all random operations during the program's runtime

In [ ]:
# @title Virtual Agent Mode
# @markdown If you'd like to run AsTRiQue in virtual agent mode, run this cell.

config.USE_VIRTUAL_AGENT = True

# @markdown You can choose which participant you'd like to simulate:

config.PARTICIPANT_TO_MODEL = "p03"  # @param ["p01", "p02", "p03", "p04", "p05", "p06", "p07", "p08", "p09", "p10", "p11", "p12", "p13", "p14", "p15", "p16", "p17", "p18", "p19", "p20", "p21", "p22", "p23", "p24", "p25", "p26", "p27", "p28", "p29", "p30", "p31"]
# @markdown participant ID to simulate
# @markdown <hr>

astrique_module.run(config)

In [ ]:
# @title Live Participant Mode
# @markdown If you'd like to run AsTRiQue in live participant mode, run this cell.

config.USE_VIRTUAL_AGENT = False


def play_sound(filename: str):
    """
    Plays a sound file, choosing the appropriate method based on environment.
    """
    if IN_COLAB:
        display(Audio(filename, autoplay=True))
    else:
        playsound(filename)


def query_participant_classification(filename: str, *args):
    """
    Plays audio and waits for human response, then returns the selected label's mapped value.
    Returns None if the user presses "exit".
    """
    filepath = os.path.join(config.AUDIO_FOLDER, filename)

    state = {"done": False, "selected": None, "exit": False}

    def make_click_fn(label=None, special=None):
        def fn(btn):
            if special == "exit":
                state["exit"] = True
                state["done"] = True
            elif special == "replay":
                play_sound(filepath)
            else:
                state["selected"] = label
                state["done"] = True

        return fn

    # create classification buttons
    label_buttons = [Button(description=l) for l in config.LABEL_MAPPING]
    for btn in label_buttons:
        btn.on_click(make_click_fn(label=btn.description))

    # create control buttons
    replay_btn = Button(description="replay")
    exit_btn = Button(description="exit")
    replay_btn.on_click(make_click_fn(special="replay"))
    exit_btn.on_click(make_click_fn(special="exit"))

    # display buttons
    controls = VBox([HBox(label_buttons), HBox([replay_btn, exit_btn])])
    display(controls)

    # play sound at the beginning
    play_sound(filepath)

    # wait for user input
    with ui_events() as poll:
        while not state["done"]:
            poll(10)
            time.sleep(0.1)

    # clean up
    controls.close()

    if state["exit"]:
        raise KeyboardInterrupt("Exited on user request.")

    selected = state["selected"]
    mapped = config.LABEL_MAPPING[selected]

    return mapped


astrique_module.run(config, query_participant_classification)